<a href="https://colab.research.google.com/github/takiyama0617/technical_analysis/blob/master/%E6%9C%89%E5%A0%B1%E3%82%AD%E3%83%A3%E3%83%83%E3%83%81%E3%83%A3%E3%83%BC%E3%81%8B%E3%82%89%E8%B2%A1%E5%8B%99%E6%83%85%E5%A0%B1%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = f'/content/drive/My Drive/workspace/technical_analysis/data/'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 1.有報キャッチャーからURLを取得
XBRLのURLリスト dat_download{ str_period }.csv が吐き出される

In [0]:
import os
import csv
import time
import re
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
from datetime import datetime, timedelta


class yuho_catcher():
    def __init__(self, since, until, base_dir=None):
        self.csv_tag = ['id', 'title', 'cd', 'url', 'update']
        self.encode_type = 'utf-8'
        self.wait_time = 2  # 間隔が短いと制限がかかる
        self.base_url = 'http://resource.ufocatch.com/atom/edinetx/'
        self.namespace = '{http://www.w3.org/2005/Atom}'
        self.out_of_since = False
        self.since = since
        self.until = until
        self.file_info_str = since.strftime(
            '_%y%m%d_') + until.strftime('%y%m%d')
        self.base_path = f'{ os.getcwd() if base_dir==None else base_dir }'

    def get_link_info_str(self, ticker_symbol):
        url = self.base_url + str(ticker_symbol)
        count, retry = 0, 3
        while True:
            try:
                response = requests.get(url)
                return response.text
            except Exception:
                print(f'{ticker_symbol} のアクセスに失敗しました。[ {count} ]')
                if count < retry:
                    count += 1
                    time.sleep(3)
                    continue
                else:
                    raise

    def parse_xml(self, string):
        ET_tree = ET.fromstring(string)
        ET.register_namespace('', self.namespace[1:-1])
        return ET_tree

    def get_link(self, tree):
        yuho_dict = {}
        # xmlのentry毎にfor
        for el in tree.findall('.//'+self.namespace+'entry'):
            title = el.find(self.namespace+'title').text
            if not self.is_yuho(title):
                continue
            updated = el.find(self.namespace+'updated').text
            checked = self.time_check(updated)
            if not checked['until']:
                continue
            if not checked['since']:
                self.out_of_since = True
                return yuho_dict
            _id = el.find(self.namespace+'id').text
            links = el.findall('./'+self.namespace+'link[@type="text/xml"]')
            for link in links:
                if '.xbrl' in link.attrib['href'] and 'PublicDoc' in link.attrib['href']:
                    url = link.attrib['href']
                    break
                else:
                    continue
            else:
                continue
            cd = re.sub(r'^【(\w+)】.*', r"\1", title)
            yuho_dict[_id] = {'id': _id, 'title': title,
                              'cd': cd, 'url': url, 'update': updated}
        return yuho_dict

    def is_yuho(self, title):
        if u'有価証券報告書' in str(title) and u'株式会社' in str(title) and u'内国投資信託受益証券' not in str(title):
            return True
        return False

    def time_check(self, update):
        updated_time = datetime.strptime(update, '%Y-%m-%dT%H:%M:%S+09:00')
        return {'since': updated_time >= self.since, 'until': updated_time < self.until}

    def dump_file(self, file, info_dict, tag, encode_type):
        with open(os.path.join(self.base_path, file), 'w', encoding=encode_type) as of:
            writer = csv.DictWriter(of, tag, lineterminator='\n')
            writer.writeheader()
            for key in info_dict:
                writer.writerow(info_dict[key])

    def craete_xbrl_url_json_each_symbols(self, list_symbols):
        print(
            f'since:{ self.since } ,until:{ self.until } ({ self.file_info_str })')
        i, result_dict = 0, {}
        for t_symbol in tqdm(list_symbols):
            response_string = self.get_link_info_str(f'query/{t_symbol}')
            ET_tree = self.parse_xml(response_string)
            info_dict = self.get_link(ET_tree)
            if len(info_dict) > 0:
                for key in info_dict:
                    result_dict[key] = info_dict[key]
                self.dump_file(
                    f'dat_download{ self.file_info_str }.csv', result_dict, self.csv_tag, self.encode_type)
            time.sleep(self.wait_time)
            i += 1
        print('complete a download!!')

    def craete_xbrl_url_json_from_latest(self):
        print(
            f'since:{ self.since } ,until:{ self.until } ({ self.file_info_str })')
        i, result_dict = 0, {}
        while True:
            page = 1 + i
            print(f'page{page}, loading...')
            response_string = self.get_link_info_str(page)
            ET_tree = self.parse_xml(response_string)
            info_dict = self.get_link(ET_tree)
            if len(info_dict) > 0:
                for key in info_dict:
                    result_dict[key] = info_dict[key]
                self.dump_file(
                    f'dat_download{ self.file_info_str }.csv', result_dict, self.csv_tag, self.encode_type)
            time.sleep(self.wait_time)
            if self.out_of_since:
                break
            i += 1
        print('complete a download!!')


In [3]:
from datetime import datetime

since = datetime.strptime('2020-04-01' ,'%Y-%m-%d')
until = datetime.strptime('2020-05-03' ,'%Y-%m-%d')
yuho = yuho_catcher( since ,until, base_dir=BASE_DIR)
yuho.craete_xbrl_url_json_from_latest()
# yuho.craete_xbrl_url_json_each_symbols( [7203] )

since:2020-04-01 00:00:00 ,until:2020-05-03 00:00:00 (_200401_200503)
page1, loading...
page2, loading...
page3, loading...
page4, loading...
page5, loading...
page6, loading...
page7, loading...
page8, loading...
page9, loading...
page10, loading...
page11, loading...
page12, loading...
complete a download!!


# 2.XBRLのURLリストからファイルをダウンロードし、CSV変換

dat_download_{from}_{to}.csvからXBRLファイルをダウンロード

ダウンロードしたXBRLファイルから必要情報を抜き出し、eggs_{from}_{to}.csvを作成する



In [4]:
!pip install python-xbrl

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for python-xbrl: filename=python_xbrl-1.1.1-py2.py3-none-any.whl size=10622 sha256=0e88460cb41c63aedfa17fee58c96d5410fcc86fa108a2dd20392a148d716aed
  Stored in directory: /root/.cache/pip/wheels/b2/a1/74/4c777bfc9b209ea0f28328facc688fe57cc9a4dd38a5756412
  Created wheel for ordereddict: filename=ordereddict-1.1-cp36-none-any.whl size=3554 sha256=0b33bbb89f5b0d680ce0b1068d4486a2a3d683e705875c61ef13e9e9a7d7785f
  Stored in directory: /root/.cache/pip/wheels/78/d3/a8/9f52c7a389a0ffba6f575a3886b5f7a55461fb6ec34aa5fd38
Successfully built python-xbrl ordereddict


In [0]:
import os
import re
import csv
import time
import urllib
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from xbrl import XBRLParser

default_tag = ['file_nm', 'element_id', 'amount']
custom_tag = ['unit_ref', 'decimals', 'contextref']
encode_type = 'utf-8'


class XbrlParser(XBRLParser):
    def __init__(self, info_dict):
        self.info_dict = info_dict
        self.xbrl_url = info_dict['url']
        xbrl_filename = self.xbrl_url.split('/')[-1]
        directory_path = f'{ os.getcwd() }/xbrl_files'
        self.__make_directory(directory_path)
        self.xbrl_filepath = f'{ directory_path }/{ xbrl_filename }.xbrl'

    def __make_directory(self, dir_name):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)

    def __download_xbrl_file(self):
        counter, retry = 0, 3
        while True:
            try:
                urllib.request.urlretrieve(self.xbrl_url, self.xbrl_filepath)
                return True
            except Exception as e:
                if counter < retry:
                    print(f'XBRLファイルのDLに失敗しました（{ counter }）{ self.xbrl_url }')
                    time.sleep(3)
                    counter += 1
                else:
                    print('上手く行かないので次の処理に移ります')
                    return False

    def parse_xbrl(self):
        DL_is_succeeded = self.__download_xbrl_file()
        if DL_is_succeeded:
            with open(self.xbrl_filepath, 'r', encoding='utf-8') as of:
                xbrl = XBRLParser.parse(of)  # beautiful soup type object
        else:
            return
        result_dicts = {}
        _idx = 0
        name_space = 'jp*'
        for node in xbrl.find_all(name=re.compile(name_space+':*')):
            if self.ignore_pattern(node):
                continue
            row_dict = {}
            row_dict['file_nm'] = self.xbrl_filepath.rsplit(os.sep, 1)[1]
            row_dict['element_id'] = node.name
            row_dict['amount'] = node.string
            for tag in custom_tag:
                row_dict[tag] = self.get_attrib_value(node, tag)
            result_dicts[_idx] = row_dict
            _idx += 1
        return result_dicts

    def ignore_pattern(self, node):
        if 'xsi:nil' in node.attrs:
            if node.attrs['xsi:nil'] == 'true':
                return True
        if not isinstance(node.string, str):
            return True  # 結果が空の場合は対象外にする
        if str(node.string).find(u'\n') > -1:
            return True  # 改行コードが結果にある場合対象外にする
        if u'textblock' in str(node.name):
            return True  # text文字列は解析に利用できないため、対象外
        return False

    def get_attrib_value(self, node, attrib):
        if attrib in node.attrs.keys():
            return node.attrs[attrib]
        else:
            return None


class parse_operator():
    def __init__(self, df, str_period, base_dir):
        self.df = df
        self.str_period = str_period
        self.base_path = base_dir

    def __dump_file(self, writer, info_dicts):
        _idx = 0
        while _idx < len(info_dicts):
            row_dict = info_dicts[_idx]
            writer.writerow(row_dict)
            _idx += 1

    def xbrl_to_csv(self):
        file = f'eggs_{ self.str_period }.csv'
        with open(os.path.join(self.base_path, file), 'w', encoding=encode_type) as of:
            resultCsvWriter = csv.DictWriter(
                of, default_tag + custom_tag, lineterminator='\n')
            resultCsvWriter.writeheader()
            for i in tqdm(range(len(self.df.index))):
                row = self.df.iloc[i]
                xp = XbrlParser(row.to_dict())
                info_dicts = xp.parse_xbrl()
                self.__dump_file(resultCsvWriter, info_dicts)
        print('completed conversions!!')


def xbrl_operator(start, end, loop_month=1, restart=0, base_dir=None):
    big_period = start.strftime('%y%m%d_')+end.strftime('%y%m%d')
    base_path = f'{ os.getcwd() if base_dir==None else base_dir }'
    file = f'dat_download_{ big_period }.csv'
    info_df = pd.read_csv(os.path.join(base_path, file),
                          parse_dates=['update'])
    since = start + relativedelta(months=(restart))
    until = start + relativedelta(months=(loop_month + restart))
    until = until if until < end else end

    while True:
        target_df = info_df.loc[(info_df['update'] >= since) & (
            info_df['update'] < until)]
        small_period = since.strftime('%y%m%d_')+until.strftime('%y%m%d')
        print(f'since:{ since } ,until:{ until } ({ small_period })')
        xbrl_parse_operator = parse_operator(
            target_df, small_period, base_path)
        xbrl_parse_operator.xbrl_to_csv()
        since = until
        if since >= end:
            break
        until = until + relativedelta(months=loop_month)
        until = until if until < end else end


In [6]:
from datetime import datetime

start = datetime.strptime('2020-04-01 +0900', '%Y-%m-%d %z')
end = datetime.strptime('2020-05-03 +0900', '%Y-%m-%d %z')
xbrl_operator(start, end, 3, base_dir=BASE_DIR)


since:2020-04-01 00:00:00+09:00 ,until:2020-05-03 00:00:00+09:00 (200401_200503)


100%|██████████| 124/124 [09:20<00:00,  4.52s/it]

completed conversions!!


# 3.CSVから必要なデータを抽出

In [0]:
import re
import pandas as pd
from tqdm import tqdm


def get_dict_edinet_codes(DIR, index='証券コード'):
    import codecs
    file_path = f'{DIR}EdinetcodeDlInfo.csv'
    # ここに掲載されている証券コードは「7203」→「72030」で表記されている
    with codecs.open(file_path, "r", "Shift-JIS", "ignore") as file:
        df = pd.read_csv(file, skiprows=[0], usecols=[
                         'ＥＤＩＮＥＴコード', '提出者名', '証券コード', '提出者業種'])
    df = df.loc[df['証券コード'] > 0, :]
    df['証券コード'] = df['証券コード'] / 10
    df['証券コード'] = df['証券コード'].astype(int)
    result = df.set_index(index).T.to_dict()
    print(f'対象(EDINET)： { len( result ) }銘柄')
    return result


def make_element_ids_csv(egg_file_name, base_dir=None):
    base_path = f'{ os.getcwd() if base_dir==None else base_dir }'
    result_file_name = f'element_ids ( { egg_file_name } ).csv'
    df_egg = pd.read_csv(os.path.join(
        base_path, egg_file_name)).drop_duplicates()
    df_egg.loc[:, 'element_id'].drop_duplicates().to_csv(
        os.path.join(base_path, result_file_name))


def make_sample_data_csv(egg_file_name, edinet_code, base_dir=None):
    base_path = f'{ os.getcwd() if base_dir==None else base_dir }'
    result_file_name = f'sample_data ( { egg_file_name } { edinet_code } ).csv'
    df_egg = pd.read_csv(os.path.join(
        base_path, egg_file_name)).drop_duplicates()
    check1 = df_egg[(df_egg['file_nm'].str.contains(edinet_code))]
    if len(check1) > 0:
        check1.to_csv(os.path.join(base_path, result_file_name))
    else:
        print(f'{ edinet_code }のデータはありませんでした')


class eggs_operator():
    def __init__(self, list_eggs, dict_codes, dict_cols, result_file_name='com_indices.csv', base_dir=None):
        self.base_path = f'{ os.getcwd() if base_dir==None else base_dir }'
        self.result_file_name = result_file_name
        self.list_df_eggs = [pd.read_csv(os.path.join(
            self.base_path, egg_file_name)).drop_duplicates() for egg_file_name in list_eggs]
        self.dict_codes = dict_codes
        self.dict_cols = dict_cols

    def __get_element(self, df, col):
        element_ids = self.dict_cols[col]['element_id']
        if col in ['会社名', '提出書類', '提出日', '年度開始日', '年度終了日']:
            check1 = df[df['element_id'].str.contains(element_ids[0].lower())]
            if len(check1) == 1:
                return check1['amount'].values[0]
            else:
                return 0
        else:
            contextref = self.dict_cols[col]['contextref']
            for element_id in element_ids:
                check1 = df[df['element_id'].str.contains(element_id.lower())]
                check2 = check1[check1['contextref']
                                == contextref].copy()  # 連結
                if len(check2) == 1:
                    return check2['amount'].values[0]
                elif len(check2) > 1:
                    check2['str_len'] = check2['element_id'].apply(
                        lambda x: len(str(x)))
                    return check2.loc[check2['str_len'] == check2['str_len'].min(), 'amount'].values[0]
            for element_id in element_ids:
                check1 = df[df['element_id'].str.contains(element_id.lower())]
                check2 = check1[check1['contextref'] ==
                                f'{ contextref }_NonConsolidatedMember'].copy()  # 個別
                if len(check2) == 1:
                    return check2['amount'].values[0]
                elif len(check2) > 1:
                    check2['str_len'] = check2['element_id'].apply(
                        lambda x: len(str(x)))
                    return check2.loc[check2['str_len'] == check2['str_len'].min(), 'amount'].values[0]
            return 0

    def get_elements(self):
        com_indices = pd.DataFrame()
        for df_eggs in self.list_df_eggs:
            file_nms = df_eggs[df_eggs['element_id'] == 'jpcrp_cor:companynamecoverpage'].drop_duplicates()[
                'file_nm'].values
            for file_nm in tqdm(file_nms):
                edinet_code = re.search(r'E[0-9]{5}', file_nm).group(0)
                if not edinet_code in self.dict_codes:
                    continue
                df_target = df_eggs[df_eggs['file_nm'] == file_nm]
                data = {col: self.__get_element(
                    df_target, col) for col in self.dict_cols}
                data['証券コード'] = self.dict_codes[edinet_code]['証券コード']
                data['業種'] = self.dict_codes[edinet_code]['提出者業種']
                data['訂正'] = 1 if '訂正' in data['提出書類'] else 0
                data['file_nm'] = file_nm
                raw = pd.DataFrame(data, index=[edinet_code])
                com_indices = pd.concat([com_indices, raw])
        com_indices.to_csv(os.path.join(self.base_path, self.result_file_name))


In [0]:
# eggに含まれている element_id の一覧を取得する
make_element_ids_csv(f'eggs_200401_200503.csv', base_dir=BASE_DIR)


In [0]:
# 特定の企業の全データを取得する
sample_code_list = ['E02293']
for edinet_code in sample_code_list:
    make_sample_data_csv(f'eggs_200401_200503.csv',
                         edinet_code, base_dir=BASE_DIR)


In [23]:
# EDINETコードリストを読み込む
dict_codes = get_dict_edinet_codes( BASE_DIR ,'ＥＤＩＮＥＴコード' )
dict_codes

対象(EDINET)： 3790銘柄


{'E00004': {'提出者名': 'カネコ種苗株式会社', '提出者業種': '水産・農林業', '証券コード': 1376},
 'E00006': {'提出者名': '株式会社\u3000サカタのタネ', '提出者業種': '水産・農林業', '証券コード': 1377},
 'E00008': {'提出者名': 'ホクト株式会社', '提出者業種': '水産・農林業', '証券コード': 1379},
 'E00009': {'提出者名': '株式会社アクシーズ', '提出者業種': '水産・農林業', '証券コード': 1381},
 'E00010': {'提出者名': '株式会社ホーブ', '提出者業種': '水産・農林業', '証券コード': 1382},
 'E00011': {'提出者名': '住友林業株式会社', '提出者業種': '建設業', '証券コード': 1911},
 'E00012': {'提出者名': '株式会社\u3000極洋', '提出者業種': '水産・農林業', '証券コード': 1301},
 'E00014': {'提出者名': '日本水産株式会社', '提出者業種': '水産・農林業', '証券コード': 1332},
 'E00015': {'提出者名': 'マルハニチロ株式会社', '提出者業種': '水産・農林業', '証券コード': 1333},
 'E00017': {'提出者名': '株式会社ホウスイ', '提出者業種': '卸売業', '証券コード': 1352},
 'E00021': {'提出者名': '三菱マテリアル株式会社', '提出者業種': '非鉄金属', '証券コード': 5711},
 'E00023': {'提出者名': '住友金属鉱山株式会社', '提出者業種': '非鉄金属', '証券コード': 5713},
 'E00024': {'提出者名': '三井金属鉱業株式会社', '提出者業種': '非鉄金属', '証券コード': 5706},
 'E00026': {'提出者名': '日本精鉱株式会社', '提出者業種': '非鉄金属', '証券コード': 5729},
 'E00027': {'提出者名': '中外鉱業株式会社', '提出者業種': '非鉄金属', '証券コード

In [0]:
##
# 読み込みの設定
#

list_eggs = [ 'eggs_200401_200503.csv' ]
result_file_name = 'com_indices_200401_200503.csv'

#（複数あるものは上のものほど優先される）
dict_cols = {
      '会社名'           : { 'element_id' : ['companynamecoverpage'] }
    , '提出書類'         : { 'element_id' : ['documenttitlecoverpage'] }
    , '提出日'           : { 'element_id' : ['filingdatecoverpage'] }
    , '年度開始日'       : { 'element_id' : ['currentfiscalyearstartdatedei'] }
    , '年度終了日'       : { 'element_id' : ['currentfiscalyearenddatedei'] }
    #---ここまで必須---
    #以下は 'contextref' が必須
    , '発行済み株式数'   : { 'element_id' : ['totalnumberofissuedsharessummaryofbusinessresults'
                                            ,'totalnumberofissuedsharescommonstocksummaryofbusinessresults'
                                            ,'totalnumberofissuedshares']
                            ,'contextref' : 'CurrentYearInstant' }
    , '営業CF'           : { 'element_id' : ['netcashprovidedbyusedinoperatingactivitiessummaryofbusinessresults'
                                            ,'cashflowsfromusedinoperatingactivitiesifrssummaryofbusinessresults'
                                            ,'CashFlowsFromUsedInOperatingActivitiesUSGAAPSummaryOfBusinessResults']
                            ,'contextref' : 'CurrentYearDuration' }
    , '財務CF'           : { 'element_id' : ['netcashprovidedbyusedinfinancingactivitiessummaryofbusinessresults'
                                            ,'cashflowsfromusedinfinancingactivitiesifrssummaryofbusinessresults'
                                            ,'CashFlowsFromUsedInFinancingActivitiesUSGAAPSummaryOfBusinessResults']
                            ,'contextref' : 'CurrentYearDuration' }
    , '投資CF'           : { 'element_id' : ['netcashprovidedbyusedininvestingactivitiessummaryofbusinessresults'
                                            ,'cashflowsfromusedininvestingactivitiesifrssummaryofbusinessresults'
                                            ,'CashFlowsFromUsedInInvestingActivitiesUSGAAPSummaryOfBusinessResults']
                            ,'contextref' : 'CurrentYearDuration' }
    , '純利益'           : { 'element_id' : ['profitlossattributabletoownersofparentsummaryofbusinessresults'
                                            ,'ProfitLossAttributableToOwnersOfParentIFRSSummaryOfBusinessResults'
                                            ,'NetIncomeLossAttributableToOwnersOfParentUSGAAPSummaryOfBusinessResults'
                                            ,'netincomelosssummaryofbusinessresults']
                            ,'contextref' : 'CurrentYearDuration' }
    , '売上高'           : { 'element_id' : ['netsalessummaryofbusinessresults'
                                            ,'NetSalesIFRSSummaryOfBusinessResults'
                                            ,'RevenuesUSGAAPSummaryOfBusinessResults'
                                            ,'operatingrevenue1summaryofbusinessresults'
                                            ,'revenueifrssummaryofbusinessresults'
                                            ,'netoperatingrevenuesummaryofbusinessresults'
                                            ,'businessrevenuesummaryofbusinessresults']
                            ,'contextref' : 'CurrentYearDuration' }
    , 'BS 現金預金'      : { 'element_id' : ['cashanddeposits']
                            ,'contextref' : 'CurrentYearInstant' }
    , 'BS 負債合計'      : { 'element_id' : ['liabilities']
                            ,'contextref' : 'CurrentYearInstant' }
    }
    

In [25]:
# 実行
eggs_operator = eggs_operator( list_eggs ,dict_codes ,dict_cols ,result_file_name=result_file_name ,base_dir=BASE_DIR )
eggs_operator.get_elements()


100%|██████████| 116/116 [00:05<00:00, 20.60it/s]
